In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image

# Temperature Sensor Configurations

Explanation of below functions:  
***Continuous conversion mode:*** The device continuously performs temperature conversions in this mode, as shown in Figure 7-3, and updates the temperature result register at the end of every active conversion  
***One-shot mode:*** After the device completes a (temperature conversion) one-shot conversion, the device goes to the *low-power shutdown mode*.  

In [26]:
class TMP117():
    def __init__(self, mode, duration, CC_params, OS_params):
        self.mode = mode
        self.time_step = CC_params[0]
        self.duration = duration
        self.CC_params = CC_params
        self.OS_params = OS_params
        self.conv_cycle = CC_params[0]
        self.time = np.arange(0,self.duration,self.time_step) #time at which to collect data
        
        #self.active_times = 
    
    def getPower(self, active_times):
        standby = 1.25 / 1000 # default power when active conversion is off
        power = 0
        
        if self.mode == "CC": 
            standByCurrentConsumption = 1.25
            activeCurrentConsumption = 135
            convCycleTime = self.CC_params[0]
            num_averages = self.CC_params[1]

            activeConversionTime = num_averages*0.0155
            standbyTime = convCycleTime - activeConversionTime
            amps = ((activeCurrentConsumption*activeConversionTime) + (standByCurrentConsumption*standbyTime)) / convCycleTime
            power = (amps * 3.3) / 1000

        elif self.mode == "OS": 
            # active conversion time equals conversion cycle time
            shutdownCurrentConsumption = 1.25
            shutdownTime = convCycleTime - activeConversionTime
            num_averages = self.OS_params[0]

            activeConversionTime = num_averages*0.0155
            power = (activeCurrentConsumption * 3.3) / 1000
        
        arr = self.returnArr(active_times)
        a = np.array(arr)
        power_arr = np.where(a == 1, power, 0)
    
        return power_arr

    def returnArr(self, active_times): 
        #time = time.tolist()
        length = len(self.time)
        arr = [0] * length # creating corresponding power array to time intervals, default values 

        # check if the given start and end time is a valid value in the time array and round to nearest value 
        for times in active_times:
            start_index = int(times[0] / self.time_step) # getting index of the closest value to active times 
            end_index = int(times[1] / self.time_step)
            
            if start_index < 0 or end_index > len(self.time): 
                print("Error. Index not valid.")
                return -1
            
            for i in range(start_index, end_index+1):
                arr[i] = 1
                
        return arr

    def getData(self, active_times):
        '''
        The data in the result register is in two's complement format, 
        has a data width of 16 bits and a resolution of 7.8125 m°C.
        
        Changing the conversion cycle period also affects the temperature result update rate because the temperature 
        result register is updated at the end of every active conversion. 
        
        Storing 16-bit value at the end of each conversion cycle
        '''
        
        # 16 bits, 32 bits, 64 bits
        # 0.0155 s, 0.0155 * 2, 0.0155 * 3
        data = 0
        self.conv_cycle
        
        arr = self.returnArr(active_times) # returns array of 0, 1's, and 2's representing mode active
        data_arr = arr
        
        beginningOfActive = True
        length = len(arr)
        
        for index, mode in enumerate(arr):
            # first time sensor is turned on
            
            if mode == 0:
                data_arr[index] = data

            elif mode == 1:
                data = data + 16
                data_arr[index] = data
            
            #elif mode == 2:
                    
        return data_arr

    def Simulation(self, active_times):
        power = self.getPower(active_times)
        data = self.getData(active_times)

        return power, data

### Parameters for TMP117 (to be changed by teams)  
...add more explanation of what each input does later  
add explanation of what possible inputs including default are for each input ...  

**Continuous conversion (if selected) params**:  
**possible conversion cycle times (see column 1 of table below)**: 0.0155, 0.125, 0.25, 0.5, 1, 4, 8, 16  
**possible averaging inputs**: 0, 8, 32, 64


In [ ]:
#Image("convTimes.png")

#### TO-DO (teams set inputs)

In [27]:
mode = 'CC'
duration = 0.5 # possible inputs: ...
convCycle = 0.0155 # possible inputs: .0155, 0.125, 0.25, 0.5, 1, 4, 8, 16, see above table (column 1)
num_averages = 0 # possible inputs: 0, 8, 32, 64
CC_params = [convCycle, num_averages] # set to 0 if mode chosen is not CC
OS_params = [num_averages] # set to 0 if mode chosen is not OS

In [28]:
x = TMP117(mode, duration, CC_params, OS_params) # creating TMP117 class
active_times = [ [0.05, 0.1], [0.3, 0.4]] # on periods, have to be multiples of the time step
power, data = x.Simulation(active_times) # getting power, data
print(len(data))

33


In [ ]:
plt.plot(power);

# Accelerometer

# Magnetometer

# Thermopile